# Sports Image Classification
Dataset: https://www.kaggle.com/code/littlebughenrylee/100-sports-classification-resnet-93-yolo-98

<p>Collection of sports images covering 100 different sports.. Images are 224,224,3 jpg format. Data is separated into train, test and valid directories. Additionallly a csv file is included for those that wish to use it to create there own train, test and validation datasets.</p>

Images were gathered from internet searches. The images were scanned with a duplicate image detector program I wrote. Any duplicate images were removed to prevent bleed through of images between the train, test and valid data sets. All images were then resized to 224 X224 X 3 and converted to jpg format. A csv file is included that for each image file contains the relative path to the image file, the image file class label and the dataset (train, test or valid) that the image file resides in. This is a clean dataset.

<img src="https://t3.ftcdn.net/jpg/02/78/42/76/360_F_278427683_zeS9ihPAO61QhHqdU1fOaPk2UClfgPcW.jpg" class="center">
<style>.center {
  display: block;
  margin-left: auto;
  margin-right: auto;
  width: 50%;
}
</style>

## Importing Necessary Libraries

In [2]:
import os 
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from torch import nn as nn
from torch.utils.data import Dataset, DataLoader
import time
import torchvision
from torchvision import transforms

from torchinfo import summary
import kaggle
from plotly.subplots import make_subplots
import os
import glob
import plotly.graph_objects as go
import plotly.express as px


## Download Dataset from Kaggle
link: https://www.kaggle.com/datasets/gpiosenka/sports-classification

In [3]:
if "sports-classification" in os.listdir():
    print('Data Already Exist')
else:
  print("Downloading Data From Kaggle")
  !kaggle datasets download -d gpiosenka/sports-classification

Data Already Exist


In [4]:
# Example usage:
directory_path = os.getcwd()

## Extract Downloaded ZIP File

In [5]:
# Extract the filenames without extension
data_folder_name = glob.glob(os.path.join(directory_path, '*.zip'))
filenames_without_extension = [os.path.splitext(os.path.basename(file))[0] for file in data_folder_name]
data_folder_name

['d:\\sports-classification.zip']

In [6]:
data_folder_name = filenames_without_extension

In [7]:
data_folder_name = data_folder_name[0]

## Determine If Dataset is unzipped or not

In [8]:
if not os.path.exists(data_folder_name):
    os.makedirs(data_folder_name)
    print(f"Directory '{data_folder_name}' created.")
    !unzip sports-classification.zip -d 'sports-classification/'
else:
    print(f"Directory '{data_folder_name}' already exists.")

Directory 'sports-classification' already exists.


## Define Model Results Directory

In [9]:
MODEL_RESULTS_DIR = 'models_results'

In [10]:
if not os.path.exists(MODEL_RESULTS_DIR):
    os.makedirs(MODEL_RESULTS_DIR)
    print(f"Directory '{MODEL_RESULTS_DIR}' created.")
else:
    print(f"Directory '{MODEL_RESULTS_DIR}' already exists.")

Directory 'models_results' already exists.


## Import Dataset and get total number of classes

In [11]:
data = pd.read_csv('sports-classification/sports.csv')
NUM_CLASSES = data['class id'].nunique()
print(NUM_CLASSES)

100


## Define Device either GPU or CPU

In [12]:
# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
# hardcode :
# loss_fn -> CrossEntropyLoss
# optimizer -> Adam(lr = 0.0005)
device

'cuda'

## Define ImageDataset
The ImageDataset class is a custom dataset implementation for handling image data in PyTorch. It is designed to work with a specific directory structure where each sub-directory corresponds to a distinct class, and the images belonging to that class are stored within that sub-directory. This structure is commonly used in image classification datasets.

The class supports image data in the JPG format. 

Attributes: The class has attributes root_dir, transform, images, labels, and class_labels.

- `root_dir`: The root directory path where the image data is stored.
- `transform`: A function for image transformations (e.g., resizing, normalization).
- `images`: A list to store the file paths of all the images in the dataset.
- `labels`: A list to store the corresponding class labels (as integers) for each image.
- `class_labels`: A dictionary mapping class directory names to integer labels.

`__len__` Method: Implements the `__len__` method to return the total number of samples (images) in the dataset. It returns the length of the images list.

`__getitem__` Method: Implements the `__getitem__` method to access a specific sample (image) and its associated label by index. It loads the image from the file path using PIL, applies the specified transformation (if any), and returns the transformed image along with its label.

Overall, the ImageDataset class simplifies the process of working with image datasets in PyTorch, providing a flexible and robust solution for handling image data in machine learning projects.

In [13]:
# Raw dataset
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):

        """
        Initializes the ImageDataset object.

        Parameters:
            root_dir (str): The root directory path containing sub-directories, each representing a class.
            transform (optional, callable): A callable function to transform the images (e.g., resizing, normalization).
        """

        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_labels = {}

        # Create a mapping of class labels to integers
        self.class_labels = {}
        class_idx = 0

        # Iterate over sub-directories
        for class_dir in os.listdir(self.root_dir):
            class_dir_path = os.path.join(self.root_dir, class_dir)
            if os.path.isdir(class_dir_path):
                self.class_labels[class_dir] = class_idx
                class_idx += 1

                # Iterate over images in the sub-directory
                for img_filename in os.listdir(class_dir_path):
                    if img_filename.endswith(".jpg"):
                        img_path = os.path.join(class_dir_path, img_filename)
                        self.images.append(img_path)
                        self.labels.append(self.class_labels[class_dir])

    def __len__(self):

        """
        Returns the total number of samples in the dataset.

        Returns:
            int: The number of samples in the dataset.
        """
        
        return len(self.images)
    
    def __getitem__(self, idx):

        """
        Retrieves a sample from the dataset.

        Parameters:
            idx (int): The index of the sample to retrieve.

        Returns:
            PIL.Image.Image: The image sample.
            int: The label associated with the image.
        """

        image = Image.open(self.images[idx])        
        label = self.labels[idx]
        
        if image.mode == "L":
            image = Image.merge("RGB", (image, image, image))
        if self.transform:
            image = self.transform(image)
            
        return image, label

## PyTorch DataLoader Function for Image Classification

### Description:
The `model_dataloader` function is designed to create PyTorch data loaders for training, validation, and testing datasets in an image classification task. It uses a custom dataset, named `ImageDataset`, which is assumed to handle a specific directory structure where each sub-directory represents a distinct class, and the images belonging to that class are stored within the respective sub-directory. This function takes weights and a transformation function as input and returns three PyTorch data loaders for the training, validation, and testing datasets.

### Parameters:
- `weights`: A variable (or value) that represents the weights used in the dataset (this variable is defined outside the function).
- `transform`: A transformation function for image preprocessing, data augmentation, or resizing. The `transform` function is applied to the images loaded from the dataset.

### Data Folder Structure:
- The function assumes that the image data is stored in the "sports-classification" folder in the following structure:

        sports-classification/ <br>
        ├── train/ <br>
        ├── valid/ <br>
        └── test/ <br>



In [14]:
# pytorch dataloader
def model_dataloder(weights, transform):
    """
    Returns three PyTorch DataLoaders for training, validation, and testing.
    
    Parameters:
        weights (list): A list of weights used for data sampling in DataLoader (optional).
        transform (torchvision.transforms): Image transformation to be applied to the datasets.
        
    Returns:
        train_dataloader (DataLoader): DataLoader for the training dataset.
        val_dataloader (DataLoader): DataLoader for the validation dataset.
        test_dataloader (DataLoader): DataLoader for the test dataset.
    """
    weights = weights
    
    data_folder = "sports-classification"

    train_folder = data_folder + "/train"
    val_folder = data_folder + "/valid"
    test_folder = data_folder + "/test"

    # Images from internet
    image_urls = "testing_images"
    
    # pytorch dataset
    train_dataset = ImageDataset(train_folder, transform = transform)
    val_dataset = ImageDataset(val_folder, transform = transform)
    test_dataset = ImageDataset(test_folder, transform = transform)

    # test downloaded  images 
    custom_images = ImageDataset(image_urls, transform = transform)
    
    # pytorch dataloader
    train_dataloader = DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True)
    val_dataloader = DataLoader(dataset = val_dataset, batch_size = 32, shuffle = False)
    test_dataloader = DataLoader(dataset = test_dataset, batch_size = 32, shuffle = False)
    custom_dataloader = DataLoader(dataset = custom_images, batch_size = 32, shuffle = False)
    
    return train_dataloader, val_dataloader, test_dataloader

## Training Function

### Description:
The `train` function is used to perform one training epoch for a given model using a specified data loader. It computes the training loss and accuracy during the training process.

### Parameters:
- `model`: The PyTorch model to be trained.
- `dataloader`: The PyTorch data loader containing the training dataset.
- `loss_fn`: The loss function used to compute the training loss.
- `optimizer`: The optimizer used to update the model's parameters during training.
- `device`: The device on which to perform computations (e.g., "cuda" for GPU or "cpu" for CPU).

### Returns:
- `train_loss`: The average training loss computed over the training dataset.
- `train_acc`: The average training accuracy computed over the training dataset.

## Validation Function

### Description:
The `val` function is used to evaluate a trained model on a validation dataset. It computes the validation loss and accuracy.

### Parameters:
- `model`: The PyTorch model to be evaluated.
- `dataloader`: The PyTorch data loader containing the validation dataset.
- `loss_fn`: The loss function used to compute the validation loss.
- `device`: The device on which to perform computations (e.g., "cuda" for GPU or "cpu" for CPU).

### Returns:
- `val_loss`: The average validation loss computed over the validation dataset.
- `val_acc`: The average validation accuracy computed over the validation dataset.

## Test Accuracy Function

### Description:
The `test_accuracy_resnet` function is used to evaluate the accuracy of a trained model on a test dataset. It computes the accuracy of the model's predictions compared to the actual labels.

### Parameters:
- `model`: The PyTorch model to be evaluated.
- `dataloader`: The PyTorch data loader containing the test dataset.
- `device`: The device on which to perform computations (e.g., "cuda" for GPU or "cpu" for CPU).

### Returns:
- `accuracy`: The accuracy of the model's predictions on the test dataset, represented as a percentage.

In [34]:
# Train -> train_loss, train_acc
def train (model, dataloader, loss_fn, optimizer, device):
    train_loss, train_acc = 0, 0
    
    model.to(device)
    model.train()
    
    for batch, (x, y) in enumerate (dataloader):
        x, y = x.to(device), y.to(device)
        
        train_pred = model(x)
        
        loss = loss_fn(train_pred, y)
        train_loss = train_loss + loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_pred_label = torch.argmax(torch.softmax(train_pred, dim = 1), dim = 1)
        train_acc = train_acc + (train_pred_label == y).sum().item() / len(train_pred)
    
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    
    return train_loss, train_acc

# Val -> val_loss, val_acc
def val (model, dataloader, loss_fn, device):
    val_loss, val_acc = 0, 0
    
    model.to(device)
    model.eval()
    
    with torch.inference_mode():
        for batch, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)
            
            val_pred = model(x)
            
            loss = loss_fn(val_pred, y)
            val_loss = val_loss + loss.item()
            
            val_pred_label = torch.argmax(torch.softmax(val_pred, dim = 1), dim = 1)
            val_acc = val_acc + (val_pred_label == y).sum().item() / len(val_pred)
        
        val_loss = val_loss / len(dataloader)
        val_acc = val_acc / len(dataloader)
        
        return val_loss, val_acc
    
def test_accuracy_resnet(model,dataloader,device):
    # empty list store labels
    predict_label_list = []
    actual_label_list = []

    # eval mode
    model.eval()

    for images, labels in dataloader: 
        
        for label in labels:
            label = label.item()
            actual_label_list.append(label)
        
        for image in images:
            with torch.inference_mode():
                image = image.to(device)
                # add batch_size and device
                image = image.unsqueeze(dim = 0)
                # logits
                logits = model(image)
                # lables
                label = torch.argmax(logits).item()
                predict_label_list.append(label)

    accuracy = accuracy_score(actual_label_list, predict_label_list)
    return accuracy*100

def classify_custom_images(model,dataloader,device,df):
    
    pred_labels = []
    # eval mode
    model.eval()

    for images, labels in dataloader: 
        
        for image in images:
            with torch.inference_mode():
                image = image.to(device)
                # add batch_size and device
                image = image.unsqueeze(dim = 0)
                # logits
                logits = model(image)
                # lables
                label = torch.argmax(logits).item()
                text_label = df[df['class id']==label]['labels'].iloc[0]
                pred_labels.append(text_label)
    return pred_labels
    

## Training Loop Function

### Description:
The `training_loop` function is responsible for training a given PyTorch model on a training dataset and evaluating its performance on a validation dataset. It allows you to control the number of training epochs and implements early stopping based on the `patience` parameter. The function records and returns the training and validation losses, accuracies, and training time for each epoch. Additionally, it displays two line charts showing the loss and accuracy trends over epochs.

### Parameters:
- `model`: The PyTorch model to be trained and evaluated.
- `train_dataloader`: The PyTorch data loader containing the training dataset.
- `val_dataloader`: The PyTorch data loader containing the validation dataset.
- `device`: The device on which to perform computations (e.g., "cuda" for GPU or "cpu" for CPU).
- `epochs`: The number of epochs for training.
- `patience`: The number of consecutive epochs with no improvement in validation loss to trigger early stopping.

### Returns:
- `model_results`: A pandas DataFrame containing the results for each epoch, including training loss, training accuracy, validation loss, validation accuracy, and time taken for each epoch.
- `training_time`: The total time taken for the entire training process.

### Training and Early Stopping:
The function trains the model for the specified number of epochs. If the validation loss does not improve for `patience` consecutive epochs, early stopping is triggered, and the training process stops to avoid overfitting.

### Line Charts:
The function creates two line charts using Plotly to visualize the loss and accuracy trends over epochs for both training and validation datasets.

In [16]:
def training_loop(model, train_dataloader, val_dataloader, device, epochs, patience):
    # empty dict for restore results
    results = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}
    
    # hardcode loss_fn and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

    # variable to hold the training time
    training_time = 0.0
    epoch_run_time = []
    # loop through epochs
    for epoch in range(epochs):

         # record the start time for each epoch
        epoch_start_time = time.time()

        train_loss, train_acc = train(model = model, 
                                      dataloader = train_dataloader,
                                      loss_fn = loss_fn,
                                      optimizer = optimizer,
                                      device = device)
        
        val_loss, val_acc = val(model = model,
                                dataloader = val_dataloader,
                                loss_fn = loss_fn,
                                device = device)
        
        # record the end time for each epoch
        epoch_end_time = time.time()
        
        # calculate the time taken for this epoch
        epoch_time = epoch_end_time - epoch_start_time
        epoch_run_time.append(epoch_time)
        training_time += epoch_time
        
        # print results for each epoch
        print(f"Epoch: {epoch+1}\n"
              f"Train loss: {train_loss:.4f} | Train accuracy: {(train_acc*100):.3f}%\n"
              f"Val loss: {val_loss:.4f} | Val accuracy: {(val_acc*100):.3f}%\n"
              f"| Epoch time: {epoch_time:.2f} seconds")
        
        # record results for each epoch
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)
        
        # calculate average "val_loss" for early_stopping
        mean_val_loss = np.mean(results["val_loss"])
        best_val_loss = float("inf")
        num_no_improvement = 0
        if np.mean(mean_val_loss > best_val_loss):
            best_val_loss = mean_val_loss
        
            model_state_dict = model.state_dict()
            best_model.load_state_dict(model_state_dict)
        else:
            num_no_improvement +=1
    
        if num_no_improvement == patience:
            break
    
    # Saving Results for model
    dic = {
        "epochs": list(range(1,epochs+1)),
        'Train_loss':results["train_loss"],
        'Train_Accuracy': results['train_acc'],
        'Validation_loss':results["val_loss"],
        'Validation_Accuracy': results['val_acc'],
        'Time_Taken':epoch_run_time

    }
    model_results = pd.DataFrame(dic)
    

    # Create the figure for the chart
    fig = go.Figure()
        # Add the 'Train loss' and 'Val loss' traces as lines
    fig.add_trace(go.Scatter(x=list(range(1, len(results["train_loss"]) + 1)), 
                            y=results["train_loss"], mode='lines', name='Train loss'))
    fig.add_trace(go.Scatter(x=list(range(1, len(results["val_loss"]) + 1)), 
                            y=results["val_loss"], mode='lines', name='Val loss'))
    

    # Update the layout for better visualization
    fig.update_layout(title="Loss over Epochs",
                    xaxis_title="Epochs",
                    yaxis_title="Loss",
                    legend=dict(x=0.05, y=1.1),
                    width=800, height=400)
    
    # Create the figure for the chart
    fig2 = go.Figure()
        # Add the 'Train loss' and 'Val loss' traces as lines
    fig2.add_trace(go.Scatter(x=list(range(1, len(results["train_acc"]) + 1)), 
                            y=results["train_acc"], mode='lines', name='Train Accuracy'))
    fig2.add_trace(go.Scatter(x=list(range(1, len(results["val_acc"]) + 1)), 
                            y=results["val_acc"], mode='lines', name='Val Accuracy'))
    

    # Update the layout for better visualization
    fig2.update_layout(title="Accuracy over Epochs",
                    xaxis_title="Epochs",
                    yaxis_title="Accuracy",
                    legend=dict(x=0.05, y=1.1),
                    width=800, height=400)

    # combined_fig.show()
    fig.show()
    fig2.show()
    
    return model_results, training_time

## Function to Add Custom Layers to a PyTorch Model

### Description:
The `add_custom_layers` function is designed to modify a pre-trained PyTorch model by adding custom layers to the model's architecture. The function is useful for fine-tuning the model for a specific task, such as image classification.

### Parameters:
- `model`: The pre-trained PyTorch model to which the custom layers will be added. The model should be an instance of a PyTorch model, such as ResNet-50.

### Returns:
- `model`: The modified PyTorch model with custom layers added.

### Custom Layers:
The function first sets `requires_grad` to `False` for all parameters in the provided model. This step is typically done to freeze the weights of the pre-trained layers, preventing them from being updated during training.

Next, the function defines a custom sequence of layers using the `nn.Sequential` module. You can modify this part of the code as per your specific requirements. In the provided example, the custom layers consist of three fully connected (linear) layers followed by ReLU activation and dropout layers.

- The first fully connected layer takes as input the number of features in the last layer of the original model (`model.fc.in_features`). The number of output features for this layer is set to 1024.
- A ReLU activation function is applied after the first fully connected layer.
- A dropout layer with a dropout probability of 0.5 is added after the first ReLU activation.
- The second fully connected layer takes 1024 input features and outputs 512 features.
- Another ReLU activation and dropout layer are applied after the second fully connected layer.
- The third and final fully connected layer takes 512 input features and outputs the number of classes (assuming `NUM_CLASSES` is defined for the specific task).

Finally, the function replaces the last layer of the provided `model` (e.g., the fully connected layer of ResNet-50) with the custom layers defined above. The modified model is then returned.

In [17]:
def add_custom_layers(model):
    for param in model.parameters():
        param.requires_grad = False
    
    
    num_features = model.fc.in_features
    
    # Define your custom layers (you can modify this part as per your requirements)
    custom_layers = nn.Sequential(
        nn.Linear(num_features, 1024),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, NUM_CLASSES)  # Assuming you have defined num_classes for your specific task
    )

    # Replace the last layer of the ResNet-50 model with the custom layers
    model.fc = custom_layers

    return model


## Function to Get ResNet Data Loaders

### Description:
The `get_resnet_dataloaders` function is designed to obtain PyTorch data loaders with data augmentation for training, validation, and testing datasets. It utilizes the `model_dataloader` function and applies specific data augmentation transformations to the images to enhance the training process for ResNet models.

### Parameters:
- `model_weights`: The weights or any other variable needed for the `model_dataloader` function (this variable is defined outside the function).

### Returns:
- `resnet_train_dataloader`: The PyTorch data loader containing the training dataset with data augmentation.
- `resnet_val_dataloader`: The PyTorch data loader containing the validation dataset with data augmentation.
- `resnet_test_dataloader`: The PyTorch data loader containing the testing dataset without data augmentation.

In [18]:
def get_resnet_dataloaders(model_weights):
    
    # resnet_weight.transforms()
    resnet_transform = transforms.Compose([
        transforms.Resize(size = 232),
        transforms.ColorJitter(brightness = (0.8, 1.2)),
        transforms.RandomHorizontalFlip(p = 0.5),
        transforms.RandomRotation(degrees = 15),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])
    resnet_train_dataloader, resnet_val_dataloader, resnet_test_dataloader = model_dataloder(weights = model_weights, 
                                                                                         transform = resnet_transform
                                                                                        )
    
    return resnet_train_dataloader, resnet_val_dataloader, resnet_test_dataloader

## Define Model Performance dataframe

In [19]:
training_times = pd.DataFrame(columns = ['Model','Testing Accuracy','Training_Time(Minutes)'])
training_times.to_csv('models_results/training_time.csv',index=False)

## ResNet Model
- ResNet, short for "Residual Network," is a deep neural network architecture that was introduced by Kaiming He et al. from Microsoft Research in 2015
- The primary motivation behind ResNet was to address the problem of vanishing gradients in very deep neural networks. 
- The ResNet architecture comes in various depths, such as ResNet-18, ResNet-34, ResNet-50, ResNet-101, and ResNet-152, among others. 


The model is based on the [ResNet](https://pytorch.org/hub/pytorch_vision_resnet/)

### ResNet50 download weights and Pretrained Models

In [27]:
resnet_weight_50 = torchvision.models.ResNet50_Weights.DEFAULT

resnet_model_50 = torchvision.models.resnet50(weights = resnet_weight_50)

#### Add some custom layers

In [28]:
resnet_model_50 = add_custom_layers(resnet_model_50)
summary(resnet_model_50)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  (4,096)
│    │    └─BatchNorm2d: 3-2             (128)
│    │    └─Conv2d: 3-3                  (36,864)
│    │    └─BatchNorm2d: 3-4             (128)
│    │    └─Conv2d: 3-5                  (16,384)
│    │    └─BatchNorm2d: 3-6             (512)
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              (16,896)
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  (16,384)
│    │    └─BatchNorm2d: 3-10            (128)
│    │    └─Conv2d: 3-11                 (36,864)
│    │    └─BatchNorm2d: 3-12            (128)
│    │    └─C

#### Transform Data using model Data Loaders

In [30]:
resnet_train_dataloader, resnet_val_dataloader, resnet_test_dataloader = get_resnet_dataloaders(resnet_weight_50)

### Start Model Training

In [ ]:
# Actual training ResNet model
resnet_results,training_time = training_loop(model = resnet_model_50,
                               train_dataloader = resnet_train_dataloader,
                               val_dataloader = resnet_val_dataloader,
                               device = device,
                               epochs = 30,
                               patience = 5
                              )

Epoch: 1
Train loss: 1.3481 | Train accuracy: 60.770%
Val loss: 0.5813 | Val accuracy: 85.117%
| Epoch time: 89.49 seconds
Epoch: 2
Train loss: 0.8833 | Train accuracy: 73.870%
Val loss: 0.4373 | Val accuracy: 87.344%
| Epoch time: 69.63 seconds
Epoch: 3
Train loss: 0.6921 | Train accuracy: 79.431%
Val loss: 0.3509 | Val accuracy: 90.469%
| Epoch time: 70.79 seconds
Epoch: 4
Train loss: 0.5773 | Train accuracy: 82.715%
Val loss: 0.3294 | Val accuracy: 91.875%
| Epoch time: 72.29 seconds
Epoch: 5
Train loss: 0.4954 | Train accuracy: 85.090%
Val loss: 0.2843 | Val accuracy: 91.562%
| Epoch time: 79.76 seconds
Epoch: 6
Train loss: 0.4531 | Train accuracy: 86.309%
Val loss: 0.2832 | Val accuracy: 90.703%
| Epoch time: 72.06 seconds
Epoch: 7
Train loss: 0.4187 | Train accuracy: 87.125%
Val loss: 0.2481 | Val accuracy: 91.680%
| Epoch time: 72.93 seconds
Epoch: 8
Train loss: 0.3741 | Train accuracy: 88.513%
Val loss: 0.2713 | Val accuracy: 91.992%
| Epoch time: 72.61 seconds
Epoch: 9
Train l

### Save Model Results

In [ ]:
resnet_results.to_csv('models_results/resnet_model_50.csv',index=False)

### Calcluate Model Testing Accuracy and Training Time Measure

In [99]:
test_accuracy = test_accuracy_resnet(resnet_model_50,resnet_test_dataloader,device)
print(f"Testing Accuracy is {test_accuracy}%")

Testing Accuracy is 95.6%


In [ ]:
print(f'Model Training Time {round(training_time/60,4)} Minitues')

Model Training Time 37.3207 Minitues


### Save Test Accuracy and Training Time in csv

In [101]:
training_times = pd.read_csv('models_results/training_time.csv')
row = {'Model':'ResNet50','Testing Accuracy':test_accuracy,'Training_Time(Minutes)':round(training_time/60,4)}

# Use the loc method to add the new row to the DataFrame
training_times.loc[len(training_times)] = row
training_times.to_csv('models_results/training_time.csv',index=False)

### Save Trained Model

In [104]:
torch.save(resnet_model_50, 'trained_models/resnet_model_50.pth')

## ResNet152
### ResNet152 download weights and Pretrained Models

In [ ]:
resnet_weight_152 = torchvision.models.ResNet152_Weights.DEFAULT

resnet_model_152 = torchvision.models.resnet152(weights = resnet_weight_152)

#### Add some custom layers

In [ ]:
resnet_model_152 = add_custom_layers(resnet_model_152)
summary(resnet_model_152)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  (4,096)
│    │    └─BatchNorm2d: 3-2             (128)
│    │    └─Conv2d: 3-3                  (36,864)
│    │    └─BatchNorm2d: 3-4             (128)
│    │    └─Conv2d: 3-5                  (16,384)
│    │    └─BatchNorm2d: 3-6             (512)
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              (16,896)
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  (16,384)
│    │    └─BatchNorm2d: 3-10            (128)
│    │    └─Conv2d: 3-11                 (36,864)
│    │    └─BatchNorm2d: 3-12            (128)
│    │    └─C

#### Transform Data using model Data Loaders

In [ ]:
resnet_train_dataloader, resnet_val_dataloader, resnet_test_dataloader = get_resnet_dataloaders(resnet_weight_152)

### Start Model Training

In [ ]:
# Actual training ResNet model
resnet_results,training_time = training_loop(model = resnet_model_152,
                               train_dataloader = resnet_train_dataloader,
                               val_dataloader = resnet_val_dataloader,
                               device = device,
                               epochs = 30,
                               patience = 5
                              )

Epoch: 1
Train loss: 2.6988 | Train accuracy: 33.988%
Val loss: 0.9109 | Val accuracy: 75.156%
| Epoch time: 125.14 seconds
Epoch: 2
Train loss: 1.0857 | Train accuracy: 68.205%
Val loss: 0.5149 | Val accuracy: 85.391%
| Epoch time: 127.26 seconds
Epoch: 3
Train loss: 0.7806 | Train accuracy: 76.638%
Val loss: 0.4604 | Val accuracy: 87.500%
| Epoch time: 125.88 seconds
Epoch: 4
Train loss: 0.6444 | Train accuracy: 80.296%
Val loss: 0.3340 | Val accuracy: 89.727%
| Epoch time: 126.72 seconds
Epoch: 5
Train loss: 0.5382 | Train accuracy: 83.586%
Val loss: 0.3082 | Val accuracy: 89.492%
| Epoch time: 126.68 seconds
Epoch: 6
Train loss: 0.4919 | Train accuracy: 84.907%
Val loss: 0.3027 | Val accuracy: 89.844%
| Epoch time: 127.62 seconds
Epoch: 7
Train loss: 0.4390 | Train accuracy: 86.647%
Val loss: 0.2093 | Val accuracy: 93.359%
| Epoch time: 129.38 seconds
Epoch: 8
Train loss: 0.3952 | Train accuracy: 87.958%
Val loss: 0.2672 | Val accuracy: 91.797%
| Epoch time: 127.86 seconds
Epoch: 9

### Save Model Results

In [ ]:
resnet_results.to_csv('models_results/resnet_model_152.csv',index=False)

### Calcluate Model Testing Accuracy and Training Time Measure

In [35]:
test_accuracy = test_accuracy_resnet(resnet_model_152,resnet_test_dataloader,device)
print(f"Testing Accuracy is {test_accuracy}%")

Testing Accuracy is 95.19999999999999%


In [ ]:
print(f'Model Training Time {round(training_time/60,4)} Minitues')

Model Training Time 65.0694 Minitues


### Save Test Accuracy and Training Time in csv

In [108]:
training_times = pd.read_csv('models_results/training_time.csv')
row = {'Model':'ResNet50','Testing Accuracy':test_accuracy,'Training_Time(Minutes)':round(training_time/60,4)}
# Use the loc method to add the new row to the DataFrame
training_times.loc[len(training_times)] = row
training_times.to_csv('models_results/training_time.csv',index=False)

In [37]:
training_times = pd.read_csv('models_results/training_time.csv')
training_times.head()

,Model,Testing Accuracy,Training_Time(Minutes)
0,ResNet50,95.6,37.3207
1,ResNet152,95.2,65.0694


### Save Trained Model

In [ ]:
torch.save(resnet_model_152, 'trained_models/resnet_model_152.pth')

## ResNet101
### ResNet101 download weights and Pretrained Models


In [40]:
resnet_weight_101 = torchvision.models.ResNet101_Weights.DEFAULT

resnet_model_101 = torchvision.models.resnet101(weights = resnet_weight_101)

#### Add some custom layers

In [20]:
resnet_model_101 = add_custom_layers(resnet_model_101)
summary(resnet_model_101)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  4,096
│    │    └─BatchNorm2d: 3-2             128
│    │    └─Conv2d: 3-3                  36,864
│    │    └─BatchNorm2d: 3-4             128
│    │    └─Conv2d: 3-5                  16,384
│    │    └─BatchNorm2d: 3-6             512
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              16,896
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  16,384
│    │    └─BatchNorm2d: 3-10            128
│    │    └─Conv2d: 3-11                 36,864
│    │    └─BatchNorm2d: 3-12            128
│    │    └─Conv2d: 3-13               

#### Transform Data using model Data Loaders

In [42]:
resnet_train_dataloader, resnet_val_dataloader, resnet_test_dataloader = get_resnet_dataloaders(resnet_weight_101)

### Start Model Training

In [85]:
# Actual training ResNet model
resnet_results,training_time = training_loop(model = resnet_model_101,
                               train_dataloader = resnet_train_dataloader,
                               val_dataloader = resnet_val_dataloader,
                               device = device,
                               epochs = 30,
                               patience = 5
                              )

Epoch: 1
Train loss: 2.7710 | Train accuracy: 31.943%
Val loss: 0.9804 | Val accuracy: 74.062%
| Epoch time: 84.57 seconds
Epoch: 2
Train loss: 1.1481 | Train accuracy: 66.598%
Val loss: 0.6465 | Val accuracy: 80.234%
| Epoch time: 94.07 seconds
Epoch: 3
Train loss: 0.8434 | Train accuracy: 74.818%
Val loss: 0.4742 | Val accuracy: 85.820%
| Epoch time: 93.41 seconds
Epoch: 4
Train loss: 0.6943 | Train accuracy: 78.922%
Val loss: 0.3933 | Val accuracy: 88.164%
| Epoch time: 93.74 seconds
Epoch: 5
Train loss: 0.6055 | Train accuracy: 81.709%
Val loss: 0.3505 | Val accuracy: 90.312%
| Epoch time: 94.25 seconds
Epoch: 6
Train loss: 0.5515 | Train accuracy: 83.257%
Val loss: 0.3462 | Val accuracy: 89.727%
| Epoch time: 94.04 seconds
Epoch: 7
Train loss: 0.4921 | Train accuracy: 85.255%
Val loss: 0.2975 | Val accuracy: 91.094%
| Epoch time: 94.33 seconds
Epoch: 8
Train loss: 0.4489 | Train accuracy: 86.229%
Val loss: 0.3028 | Val accuracy: 90.195%
| Epoch time: 93.86 seconds
Epoch: 9
Train l

### Save Model Results

In [86]:
resnet_results.to_csv('models_results/resnet_model_101.csv',index=False)

### Calcluate Model Testing Accuracy and Training Time Measure

In [40]:
resnet_model_101 = torch.load('trained_models/resnet_model_152.pth')
test_accuracy = test_accuracy_resnet(resnet_model_101,resnet_test_dataloader,device)
print(f"Testing Accuracy is {test_accuracy}%")

Testing Accuracy is 94.0%


In [88]:
print(f'Model Training Time {round(training_time/60,4)} Minitues')

Model Training Time 58.9779 Minitues


### Save Test Accuracy and Training Time in csv

In [122]:
row = {'Model':'ResNet101','Testing Accuracy':test_accuracy,'Training_Time(Minutes)':round(training_time/60,4)}

# Use the loc method to add the new row to the DataFrame
training_times.loc[len(training_times)] = row
training_times.to_csv('models_results/training_time.csv',index=False)

In [43]:
training_times = pd.read_csv('models_results/training_time.csv')
training_times.head()

,Model,Testing Accuracy,Training_Time(Minutes)
0,ResNet50,95.6,37.3207
1,ResNet152,95.2,65.0694
2,ResNet101,94.0,58.9779


### Save Trained Model

In [94]:
torch.save(resnet_model_101, 'trained_models/resnet_model_101.pth')